In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

## Define locations

In [103]:
loc_output = os.path.join('..', 'data', 'generated_forecasts.xlsx')

## Define some general functions

In [97]:
def create_demand(quarters, seasonality, season_speed, trendline, noise):

    demand = seasonality * np.sin(quarters * season_speed) \
                + trendline * quarters\
                + noise * np.random.normal(size = (len(quarters)))
    return np.maximum(0, np.round(demand).astype(int))

def generate_forecasts(value, quarters_ahead, noise):

    return np.maximum(0, np.round(value + noise * np.random.normal(size = (quarters_ahead))).astype(int) )

## Select some parameters

In [98]:
quarters = np.arange(1, 21, step = 1)
quarters_ahead = 6
seasonality = 15
season_speed = 1/2
trendline = 5
noise = 15
noise_forecasts = 10

## Generate actuals & forecasts

In [99]:
# Step 1: generate actual sales
actuals = create_demand(quarters, seasonality, season_speed, trendline, noise)

# Step 2: generate forecasts and put everything together
dfs = list()
for quarter, actual in zip(quarters, actuals):
    num_forecasts = min(quarters_ahead, quarter)
    forecasts = generate_forecasts(actual, num_forecasts - 1, noise_forecasts)
    forecasts = np.append(forecasts, actual)
    
    dfs.append(pd.DataFrame({
        'quarter' : np.arange(quarter - num_forecasts + 1, quarter + 1),
        'quarter_forecasted' : quarter,
        'forecast' : forecasts
    }))


df = pd.concat(dfs).sort_values(['quarter', 'quarter_forecasted'])

## Save actuals to a file

In [104]:
df.to_excel(loc_output, index = False)

In [105]:
loc_output

'..\\data\\generated_forecasts.xlsx'